# I. Introduction

Our group uses LightGBM for fraud detection. 


In [0]:
import os 
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
from multiprocessing import Pool
from IPython.display import display
import lightgbm as lgb
import zipfile
import datetime
import time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# II. LGBM

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
zf_df = zipfile.ZipFile('/content/drive/My Drive/train_1204.csv.zip')
df = pd.read_csv(zf_df.open('train_1204.csv'))
df.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V11,V12,V13,V14,V15,V16,V19,V20,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V40,V44,V45,V46,V47,V53,V54,V55,V56,V61,V62,V65,V66,V67,V70,V75,V76,V77,V78,V81,V82,V83,V86,V87,V88,V98,V99,V100,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V130,V131,V135,V136,V137,V169,V170,V171,V172,V173,V174,V175,V176,V180,V181,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V205,V207,V208,V209,V210,V214,V215,V216,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V234,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V264,V267,V268,V270,V271,V272,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V297,V298,V299,V300,V301,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,368.164292,150.0,1,142.0,1,315.0,87.0,19.000000,-1,-1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,13.000000,44.308274,52.418364,13.000000,13.000000,0.000000,1,1,1,2,0,1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,-5.0,1102.0,0.0,0.0,100.0,-1,52.0,-1,-1,166.0,266.0,507.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2987001,0,86401,29.0,4,2755,404.000000,150.0,2,102.0,1,325.0,87.0,973.679684,16,-1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,41.611823,0.000000,86.845376,0.000000,-104.430755,0.000000,-1,-1,-1,0,1,1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,1102.0,0.0,0.0,100.0,-1,52.0,-1,-1,166.0,266.0,507.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2987002,0,86469,59.0,4,4663,490.000000,150.0,3,166.0,2,330.0,87.0,287.000000,35,-1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,35.007066,0.000000,75.318543,0.000000,315.000000,315.000000,1,1,1,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

In [0]:
X = df.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = df.sort_values('TransactionDT')['isFraud']


In [0]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [0]:
folds = TimeSeriesSplit(n_splits=5)

aucs = list()
recall = list()
precision = list()
f1 = list()

feature_importances = pd.DataFrame()
feature_importances['feature'] = X.columns

training_start_time = time.time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
    start_time = time.time()
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
    clf = lgb.train(params, trn_data, 10000, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval=1000, 
                    early_stopping_rounds=500)
    
    feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
    aucs.append(clf.best_score['valid_1']['auc'])
    
    lgb_prediction = clf.predict(X.iloc[test_idx],type = "probs")

    y_lab = [1 if i > 0.027 else 0 for i in lgb_prediction ]
    print("Recall: ",recall_score(y.iloc[test_idx], y_lab))
    print("Precision: ",precision_score(y.iloc[test_idx], y_lab))
    print("F1 Score: ", f1_score(y.iloc[test_idx], y_lab))

    recall.append(recall_score(y.iloc[test_idx], y_lab))
    precision.append(precision_score(y.iloc[test_idx], y_lab))
    f1.append(f1_score(y.iloc[test_idx], y_lab))
    
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time.time() - start_time))))
print('-' * 30)
print('Training has finished.')
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time.time() - training_start_time))))
print('Mean AUC:', np.mean(aucs))
print('Mean Recall:', np.mean(recall))
print('Mean Precision:', np.mean(precision))
print('Mean F1 Score:', np.mean(f1))
print('-' * 30)

Training on fold 1
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999998	valid_1's auc: 0.949672
Early stopping, best iteration is:
[1302]	training's auc: 1	valid_1's auc: 0.950895
Recall:  0.7329906259449652
Precision:  0.39305983460353494
F1 Score:  0.5117162761241292
Fold 1 finished in 0:04:16.646129
Training on fold 2
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999986	valid_1's auc: 0.957469
[2000]	training's auc: 1	valid_1's auc: 0.95862
Early stopping, best iteration is:
[1729]	training's auc: 1	valid_1's auc: 0.958693
Recall:  0.7552981764415968
Precision:  0.5305521897178467
F1 Score:  0.623284189120488
Fold 2 finished in 0:10:11.276782
Training on fold 3
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999875	valid_1's auc: 0.955135
[2000]	training's auc: 1	valid_1's auc: 0.957918
Early stopping, best iteration is:
[2398]	training's auc: 1	valid_1's auc: 

# III. Future work:

- Rest of the LGBM model
- Could try some feature engineering.
- Play around with thresholds to get best model performance
- Tune the model hyperparameters
